
# Agno Teams — Concepts to Shared State (with Mermaid diagrams)

This notebook explains the core ideas for building and operating **Agno Teams** — from the basic concept of a Team to **Shared State**.  
The focus is on **how things work** and **when to use which setting**, with compact diagrams and tables.

> Note: Mermaid diagrams are included using fenced code blocks (```mermaid). They render in many viewers (GitHub, VS Code with Mermaid extension, JupyterLab with a Mermaid plugin).



## 1) What is a Team?

A **Team** is a coordinator that can delegate work to **members**. Members can be either **Agents** or **sub-Teams**.  
Think of the Team as a team lead that:
- reads the user's request,
- decides whether to **delegate** to one or more members,
- collects responses,
- and produces a final answer.

### Team Composition (Mermaid)
```mermaid
flowchart TD
    U[User] -->|input| TL[Team Leader]
    subgraph TEAM[Team]
      TL --> M1[Agent A]
      TL --> M2[Agent B]
      TL --> ST1[Sub-Team]
      subgraph ST1[Sub-Team]
        STL[Sub-Team Lead] --> SA1[Agent C]
      end
    end
    M1 --> TL
    M2 --> TL
    ST1 --> TL
    TL -->|final response| U
```

### Key properties
- **members**: list of `Agent` or `Team`
- **model**: model used by the Team leader to reason/coordinate
- **instructions/description**: shape the leader’s behavior
- **tools (optional)**: tools the *leader* can call (members manage their own tools)
- **storage & memory (optional)**: persist runs, enable history/memories
- **knowledge (optional)**: add a knowledge base for retrieval



## 2) Building Teams

Start small: define specialized members and give the Team leader simple instructions. Add extras (history, tools, knowledge) only as needed.

### When creating a Team
| Area | What to decide | Practical tip |
|---|---|---|
| Members | Which skills belong in separate agents/sub-teams | Name/role per member helps the leader route correctly |
| Leader Model | Faster vs smarter vs reasoning-heavy | Start with a capable general model; upgrade if complex delegation |
| Instructions | What “good” output means | Include output expectations (format, depth, tone) |
| Tools | Team-lead tools vs member tools | Prefer tools on members; leader tools only when truly needed |
| Storage | Where runs/state live | Enable when you need history-aware behavior |
| Memory/Knowledge | Personalization and RAG | Add later to avoid early complexity |

Minimal code is sufficient once you pick the members, leader model, and a short instruction list.



## 3) Running Teams

You typically call `Team.run()` (or the async variant) with input. The Team leader then decides whether to answer directly or delegate.

### Run Flow (Mermaid Sequence)
```mermaid
sequenceDiagram
    participant U as User
    participant T as Team Leader
    participant A as Member(s)
    U->>T: input
    T->>T: analyze & plan
    alt delegate needed
        T->>A: task (may be multiple in parallel)
        A-->>T: member result(s)
        T->>T: analyze member results
    else no delegation
        T->>T: prepare direct response
    end
    T-->>U: final output
```

### Streaming & events
- **Streaming content**: stream the final content as it is produced.  
- **Event stream**: optionally include tool calls, reasoning steps, and member events to debug/monitor live execution.

Use streaming during development to see how delegation unfolds.



## 4) How Team Execution Works

The Team has an internal **Team Leader** that may use a special tool (`delegate_task_to_members`) to send tasks to members.

### Delegation Variants
- **Default**: Leader decides *who* to delegate to and *what input* to send; then synthesizes a final answer.
- **`respond_directly=True`**: Return member responses as-is (leader skips synthesis). Useful for router patterns.
- **`determine_input_for_members=False`**: Pass the **user’s input** directly to members without leader rewriting it.
- **`delegate_task_to_all_members=True`**: Broadcast the same task to all members (often with async runs to parallelize).

### Delegation Path (Mermaid)
```mermaid
flowchart LR
    I[User Input] --> P[Leader: Plan]
    P -->|single member| M1[Member X]
    P -->|multi members| M2[Member Y]
    P -->|all members| M3[Member Z]
    M1 --> A[Leader: Analyze results]
    M2 --> A
    M3 --> A
    A --> O[Final Response]
```

**Guide**  
- Use **router style** when the user’s language or topic maps 1-1 to a member.  
- Use **broadcast** when you want parallel viewpoints or sources.  
- Use **pass-through input** when members expect structured/validated input directly.



## 5) Debugging Teams

Turn on **debug mode** to see the exact context sent to models, delegation calls, tool usage, and timing/metrics. There’s also a CLI for interactive exploration.

### Common event types (condensed)
| Category | Examples | Purpose |
|---|---|---|
| Core | RunStarted, RunContent, RunCompleted, Error | High-level lifecycle |
| Tools | ToolCallStarted/Completed | Trace tool usage and payloads |
| Reasoning | ReasoningStarted/Step/Completed | Observe chain-of-thought *signals* (structured hints) and pacing |
| Memory | MemoryUpdateStarted/Completed | Track memory writes |
| Sessions | SessionSummaryStarted/Completed | When summaries are enabled |
| Custom | Your own domain events | Attach domain telemetry |

Use event streams to pinpoint slow tools, repeated delegation, or missing instructions.



## 6) Conversation History

With storage enabled, Teams can pull past context into the leader’s prompt and/or expose tools to search it.

### Options overview
| Scope | Setting | What it does |
|---|---|---|
| Team-level | `add_history_to_context` + `num_history_runs` | Auto-include recent exchanges for continuity |
| Team-level | `read_chat_history` / `read_tool_call_history` | Add tools for on-demand lookup of prior messages/tool calls |
| Team→Members | `add_team_history_to_members` + `num_team_history_runs` | Send recent team-level history along to members during delegation |
| Cross-session | `search_session_history` + `num_history_sessions` | Allow fetching context from prior sessions |
| Size control | `max_tool_calls_from_history` | Limit how many past tool calls are injected |

### History/Data Path (Mermaid)
```mermaid
flowchart TB
    DB[(Database)]
    subgraph Leader Context Build
      H1[Recent team turns] --> C[Context]
      H2[Optional: session summary] --> C
      H3[Optional: user memories] --> C
      H4[Optional: knowledge] --> C
    end
    DB --> H1
    DB --> H2
    DB --> H3
    DB --> H4
    C --> L[Leader -> Decide/Delegate]
    L --> M[Members]
    style DB fill:#eef,stroke:#66c
```

Use the smallest history that still answers follow-ups; excessive history increases cost and may drown important details.



## 7) Team Sessions

A **session** connects a sequence of runs. Use `user_id` and `session_id` to separate multi-user, multi-session interactions.

### Session capabilities
- **Per-session continuity**: keep chat and state for a specific conversation thread.
- **Summaries**: enable automatic short summaries when histories grow.
- **Selective persistence**: control whether media, tool messages, and history are persisted.

### Session Overview (Mermaid)
```mermaid
flowchart TD
    subgraph User A
      A1[Session A1] --> A2[Session A2]
    end
    subgraph User B
      B1[Session B1]
    end
    DB[(Storage)] --- A1
    DB --- A2
    DB --- B1
    A1 -.history/memory.-> A2
```

**Tip**: Use summaries to keep context short; always prefer current conversation facts over old summaries.



## 8) Context Engineering

The leader’s **system message** is the backbone: it combines the Team’s description, instructions, member info, and coordination rules.

### System message ingredients
- **Description**: short statement of purpose
- **Instructions**: concrete do/don’t guidance and expectations
- **Expected output**: format/shape the answer
- **Member roster**: names, roles, (optional) tools
- **Extras** (opt-in): datetime, location, memories, summaries, knowledge, dependencies, session state

### Parameter quick map
| Parameter | Why it matters |
|---|---|
| `instructions`, `description`, `expected_output` | Steer behavior and formatting |
| `add_member_tools_to_context` | Show tools a member can use; helps routing |
| `respond_directly`, `delegate_task_to_all_members`, `determine_input_for_members` | Execution shape knobs |
| `add_*_to_context` flags | Opt specific signals into the system message |
| `get_member_information_tool` | Allow on-demand member metadata lookup |
| `system_message` | Full manual override when you must |
| Caching note | Stable prefix (e.g., roster + rules) can benefit from provider prompt caching |

### Context Build (Mermaid)
```mermaid
flowchart LR
    D[Description] --> S[System Message]
    I[Instructions] --> S
    E[Expected Output] --> S
    R[Member Roster] --> S
    X[Optional: Memories, Summary, State, Knowledge, Time/Loc] --> S
    U[User Input] --> C[Final Prompt to Model]
    S --> C
```

Write concise instructions first, then iterate. Move stable boilerplate earlier in the message to benefit from caching.



## 9) Input & Output

Teams can work with plain strings or **structured data** (via pydantic models). They can also produce structured results.

### Options at a glance
| Topic | What it enables | Notes |
|---|---|---|
| Plain text I/O | Fast prototyping | Least rigid |
| **Structured output** (`output_schema`) | Validated, predictable responses | Combine with tools/knowledge; supports streaming a single structured event |
| **Structured input** (team-level `input_schema`) | Validates incoming payloads | Pass dicts that validate against schema |
| Input validation (dict + `input_schema`) | Automatic checks before run | Fail early on invalid payloads |
| Parser model | Parse/normalize the main model’s output | Use a smaller model to enforce structure |
| Output model | Let one model reason, another produce final format | Handy for multimodal or strict formatting |

**Trade-off**: more structure ⇒ more reliability, but also more constraints and sometimes higher latency.



## 10) Dependencies

**Dependencies** inject dynamic variables or functions into the Team context (e.g., user profile, clock, examples, retrieved docs).

### Patterns
- **Static**: a dict of constant values
- **Dynamic**: callables that resolve at runtime
- **Add to user message**: set `add_dependencies_to_context=True` to surface the entire dependency bundle to the model

### Dependency Path (Mermaid)
```mermaid
flowchart LR
    F[Functions/Values] --> R[Resolve at run]
    R --> K[Injected into context]
    K --> L[Leader Plans/Delegates]
```

Keep dependencies focused: only inject the variables that truly influence the decision or output formatting.



## 11) Shared State

Teams and members can collaborate over a **shared `session_state`** (e.g., a shopping list). Tools receive `session_state` automatically, and updates propagate across the team.

### Key switches
| Setting | Purpose |
|---|---|
| `session_state={...}` | Initialize shared keys/values |
| `add_session_state_to_context=True` | Make state visible in prompts so models can reason about it |
| `enable_agentic_state=True` | Allow agents/teams to update state via tools |
| `share_member_interactions=True` | Provide other members with interaction logs **during the same run** |

### State Flow (Mermaid)
```mermaid
sequenceDiagram
    participant U as User
    participant T as Team Leader
    participant M as Member Tool
    participant S as session_state
    U->>T: request (e.g., add/remove item)
    T->>M: call tool with session_state
    M->>S: read/modify
    S-->>M: updated view
    M-->>T: result message
    T-->>U: confirmation + current state
```

State can also be referenced in **instructions** using placeholders (e.g., `{user_name}`) so the leader can format responses with live values.



## 12) Quick Reference

### Decision guide
```mermaid
flowchart TD
    A[User request] --> B{Single skill or multiple?}
    B -->|Single| R1[Route to one member]
    B -->|Multiple| R2[Delegate to several]
    R2 --> C{Need parallel POVs?}
    C -->|Yes| D[delegate_task_to_all_members=True]
    C -->|No| E[Default delegation]
    A --> F{Member needs raw user input?}
    F -->|Yes| G[determine_input_for_members=False]
    A --> H{Return member output verbatim?}
    H -->|Yes| I[respond_directly=True]
```

### Common pitfalls
- Overly long instructions: prefer short bullet points with specific expectations
- Too much history: start with 2–3 turns and increase only if needed
- Unclear member roles: ambiguous roles produce ping-pong delegation
- Global tools on leader when member tools suffice: keep tools close to where they’re used
- Forgetting to expose `session_state` to context when state-driven formatting is required

### Summary
- A Team is a **delegator** with finely tunable behavior knobs.  
- Use **structured input/output** when reliability matters.  
- Bring in **history, memory, knowledge, and dependencies** in measured doses.  
- **Shared State** enables coordinated, stateful workflows across members.
